# Projeto 1 - Ciência dos Dados

Nome: Paulo Kim

Nome: Rodrigo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import math

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'iPhone_X.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPhone_X.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train = train.iloc[:,:2]
train.head(5)

,Treinamento,Classificação
0,@iconredesign my prediction is the same but:\n...,2
1,still have mine but i never connected to my ph...,1
2,"iphone 11 na waste of money, just chill with x...",2
3,once the iphone 12 drops those using the iphon...,2
4,#ncat who fix’s phone ?? it’s a iphone x,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test = test.iloc[:,:2]
test.head(5)

,Teste,Classificação
0,@starrynight024 right?? that’s exactly what ha...,3
1,@gossipgirliee some of us haven't bought iphone x,2
2,@manyaww @mlgcollegefess hs without cond*m and...,1
3,very annoying that they didn't announce the ne...,2
4,@clarkvolo iphone x was released in november tho,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nós pegamos como produto o iPhone X. Para sua classificação, dividimos os tweets em:

* Propaganda/oferta (0)
* Muito irrelevante (1)
* Irrelevante (2)
* Relevante (3)
* Muito relevante (4)

In [5]:
# Função cleanup que realiza a filtragem de caracteres desnecessários
def cleanup(text):
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    text_subbed_list = text_subbed.split()
    
    for word in text_subbed_list:
        if word[0] == '@' or word[:5] == 'https':
            text_subbed_list.remove(word)
        else:
            pass
    return ' '.join(text_subbed_list)  

In [6]:
train.head()

,Treinamento,Classificação
0,@iconredesign my prediction is the same but:\n...,2
1,still have mine but i never connected to my ph...,1
2,"iphone 11 na waste of money, just chill with x...",2
3,once the iphone 12 drops those using the iphon...,2
4,#ncat who fix’s phone ?? it’s a iphone x,1


In [7]:
# Loop para realizar a limpeza
for i in range(len(train)):
    train.at[i,'Treinamento'] = cleanup(train.Treinamento[i]).lower() 
                                                                      
train

,Treinamento,Classificação
0,my prediction is the same but a13 bionic face ...,2
1,still have mine but i never connected to my ph...,1
2,iphone 11 na waste of money just chill with x ...,2
3,once the iphone 12 drops those using the iphon...,2
4,ncat who fix’s phone it’s a iphone x,1
...,...,...
1245,and now the iphone 12p just seems like a borin...,3
1246,hi there my app isnt loading any data iphone x...,4
1247,word me too i have rocking this iphone x for a...,3
1248,jetech screen protector for iphone 11 pro ipho...,0


In [8]:
# Calculando o total de palavras no conjunto dos tweets 
total_palavras = 0 
for phrase in train.Treinamento:
    total_palavras += len(phrase.split())

___
### Montagens das séries e tabelas relativas para cada classificação

Filtragem do dataframe para cada classficação, para assim montar as tabelas com as probabilidades de cada palavra dada uma classificação

(TOKKI) Aqui não gostei muito do jeito que eu fiz, queria ter feito algo mais "limpo", pode mudar se tiver uma ideia melhor 

In [9]:
# Transformando a lista de palavras da classficação Propaganda/Oferta (0) em uma série
lista_palavras_class_0 = []
palavras_class_0 = train.loc[train.Classificação == 0,:]

for phrase in palavras_class_0.Treinamento:
    lista_palavras_class_0 += phrase.split()
    
series_palavras_class_0 = pd.Series(lista_palavras_class_0)

In [10]:
# Transformando a lista de palavras da classficação Muito Irrelevante (1) em uma série
lista_palavras_class_1 = []
palavras_class_1 = train.loc[train.Classificação == 1,:]

for phrase in palavras_class_1.Treinamento:
    lista_palavras_class_1 += phrase.split()
    
series_palavras_class_1 = pd.Series(lista_palavras_class_1)

In [11]:
# Transformando a lista de palavras da classficação Irrelevante (2) em uma série
lista_palavras_class_2 = []
palavras_class_2 = train.loc[train.Classificação == 2,:]

for phrase in palavras_class_2.Treinamento:
    lista_palavras_class_2 += phrase.split()
    
series_palavras_class_2 = pd.Series(lista_palavras_class_2)

In [12]:
# Transformando a lista de palavras da classficação Relevante (3) em uma série
lista_palavras_class_3 = []
palavras_class_3 = train.loc[train.Classificação == 3,:]

for phrase in palavras_class_3.Treinamento:
    lista_palavras_class_3 += phrase.split()
    
series_palavras_class_3 = pd.Series(lista_palavras_class_3)

In [13]:
# Transformando a lista de palavras da classficação Muito Relevante (4) em uma série
lista_palavras_class_4 = []
palavras_class_4 = train.loc[train.Classificação == 4,:]

for phrase in palavras_class_4.Treinamento:
    lista_palavras_class_4 += phrase.split()
    
series_palavras_class_4 = pd.Series(lista_palavras_class_4)

In [14]:
# Calculando a quantidade de cada palavra dada a classificação
qntd_absoluta_palavra_class_0 = series_palavras_class_0.value_counts()
qntd_absoluta_palavra_class_1 = series_palavras_class_1.value_counts()
qntd_absoluta_palavra_class_2 = series_palavras_class_2.value_counts()
qntd_absoluta_palavra_class_3 = series_palavras_class_3.value_counts()
qntd_absoluta_palavra_class_4 = series_palavras_class_4.value_counts()

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [15]:
# Calculando a probabilddiade de uma palavra pertencer à respectiva classificação
P_0 = len(train.loc[train['Classificação']== 0,:]) / len(train)
P_1 = len(train.loc[train['Classificação']== 1,:]) / len(train)
P_2 = len(train.loc[train['Classificação']== 2,:]) / len(train)
P_3 = len(train.loc[train['Classificação']== 3,:]) / len(train)
P_4 = len(train.loc[train['Classificação']== 4,:]) / len(train)

In [16]:
# Smoothing
smoothing = {
    'V': 10**6,
    'alpha': 0.5
}

# Funções resposáveis em calcular as probabilidades com a aplicação do smoothing

def prob_frase_dado_class_0(frase):
    prob = 1
    for palavra in frase.split():
        try: # Verifica se a palavra está contida na série da respectiva categoria. 
            prob *= (qntd_absoluta_palavra_class_0[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_0.sum() + smoothing['alpha']*smoothing['V']) 
        except: # Caso a palavra não exista, a técnica de smoothing impede que a probabilidade resulte em zero
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_0.sum() + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_1(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_1[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_1.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_1.sum() + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_2(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_2[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_2.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(qntd_absoluta_palavra_class_2) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_3(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_3[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_3.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_3.sum() + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_4(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (qntd_absoluta_palavra_class_4[palavra] + smoothing['alpha']) / (qntd_absoluta_palavra_class_4.sum() + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (qntd_absoluta_palavra_class_4.sum() + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

In [17]:
def naive_bayes(tweet):
    p_0_dado_frase = math.log(prob_frase_dado_class_0(tweet) * P_0)
    p_1_dado_frase = math.log(prob_frase_dado_class_1(tweet) * P_1)
    p_2_dado_frase = math.log(prob_frase_dado_class_2(tweet) * P_2)
    p_3_dado_frase = math.log(prob_frase_dado_class_3(tweet) * P_3)
    p_4_dado_frase = math.log(prob_frase_dado_class_4(tweet) * P_4)
    
    # As probabilidade são guardadas em uma lista, para assim, retornar o maior valor dela
    lista_de_probabilidades = [p_0_dado_frase,
                               p_1_dado_frase,
                               p_2_dado_frase,
                               p_3_dado_frase,
                               p_4_dado_frase]
    
    #Já que a lista está ordenada de acordo com as categorias, o index da maior probabilidade será a própria classificação
    return lista_de_probabilidades.index(max(lista_de_probabilidades))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [18]:
for i in range(len(test)):
    test.at[i,'Teste'] = cleanup(test.Teste[i]).lower() 
                                                                      
test.head(10)

,Teste,Classificação
0,right that’s exactly what happened when i got ...,3
1,some of us havent bought iphone x,2
2,@mlgcollegefess hs without condm and what did ...,1
3,very annoying that they didnt announce the new...,2
4,iphone x was released in november tho,2
5,what phone do you use — iphone x max,1
6,@trevorjd14 so when ill trade in my iphone x f...,2
7,apple iphone x 64gb space grey renewed amazon ...,0
8,hi i am having problems logging in to pay invo...,3
9,the xbox one x was discontinued though so that...,2


In [19]:
test.loc[:,'NBayes'] = 0
for i in range(len(test)):
    test.iloc[i,2] = naive_bayes(test.Teste[i])
test

,Teste,Classificação,NBayes
0,right that’s exactly what happened when i got ...,3,1
1,some of us havent bought iphone x,2,1
2,@mlgcollegefess hs without condm and what did ...,1,1
3,very annoying that they didnt announce the new...,2,1
4,iphone x was released in november tho,2,1
...,...,...,...
720,mojosmartphone qi leather wireless charger htt...,0,0
721,shop anythingyoucanimagine https//tco/2xlxbref9m,0,0
722,glowing apple logo backlit led case for iphone...,0,0
723,shit crazy had this iphone x forever,2,1


In [20]:
tabela_acuracia_teste = pd.crosstab(test.Classificação, test.NBayes, normalize = True)
tabela_acuracia_teste

NBayes,0,1,2,3
Classificação,,,,
0,0.187586,0.031724,0.000000,0.000000
1,0.008276,0.088276,0.000000,0.001379
2,0.027586,0.431724,0.001379,0.013793
3,0.000000,0.148966,0.005517,0.005517
4,0.000000,0.042759,0.001379,0.004138


In [21]:
taxa_acuracia_teste = tabela_acuracia_teste.iloc[0,0] + tabela_acuracia_teste.iloc[1,1] + tabela_acuracia_teste.iloc[2,2] + tabela_acuracia_teste.iloc[3,3] 
print('A taxa de acurácia do classificador no teste é de {}%'.format(taxa_acuracia_teste.round(4)*100))

A taxa de acurácia do classificador no teste é de 28.28%


In [22]:
train.loc[:,'NBayes'] = 0
for i in range(len(train)):
    train.iloc[i,2] = naive_bayes(train.Treinamento[i])

In [23]:
tabela_acuracia_treino = pd.crosstab(train.Classificação, train.NBayes, normalize = True)
tabela_acuracia_treino

NBayes,0,1,2,3
Classificação,,,,
0,0.2104,0.0112,0.0008,0.0000
1,0.0056,0.3512,0.0000,0.0000
2,0.0024,0.1424,0.0504,0.0016
3,0.0008,0.0736,0.0000,0.0848
4,0.0000,0.0600,0.0000,0.0048


In [24]:
taxa_acuracia_treino = tabela_acuracia_treino.iloc[0,0] + tabela_acuracia_treino.iloc[1,1] + tabela_acuracia_treino.iloc[2,2] + tabela_acuracia_treino.iloc[3,3] 
print('A taxa de acurácia do classificador no teste é de {}%'.format(taxa_acuracia_treino.round(4)*100))

A taxa de acurácia do classificador no teste é de 69.67999999999999%


___
## Concluindo:

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [25]:
# dict_lista_palavras = {}

# for index_class in range(5):
#     palavras_class = train.loc[train.Classificação == index_class,:]
#     lista = []
#     for phrase in palavras_class.Treinamento:
#         lista += phrase.split()
#     dict_lista_palavras[index_class] = lista

# dt_palavras = pd.DataFrame.from_dict({ key:pd.Series(value) for key, value in dict_lista_palavras.items() })
# dt_palavras